In [1]:
import itertools as it
import numpy as np
from itertools import product
from scipy.stats import gamma
from sklearn.neighbors import KernelDensity
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.figure_factory as ff

from IPython.display import clear_output

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)


In [2]:

bases = 'ACGT'

mutations=[]

kmer= 3 ## must be odd
base_set= [bases]*kmer

for trimer in product(*base_set):
    for base in bases:
        if trimer[int(kmer / 2)] != base:
            mutations.append((''.join(trimer), base))


In [3]:
bases_probs= [.25] * 4
L= 10000
random_fasta= np.random.choice(list(bases),size=L,p= bases_probs,replace= True)
random_fasta= ''.join(random_fasta)
random_fasta[:100]

'AGCCTGAAGCGATCAGGGCGTGGTGTCCCCGCATATTAACAATAGTCGTGTCAACATCAATTAAGATTGAAAGTGATCATTCTGATCATTATCGGTAATT'

## read fasta and get mutation count

In [4]:
from functools import reduce  # forward compatibility for Python 3
import operator
import collections

def recursively_default_dict():
    return collections.defaultdict(recursively_default_dict)


####
####
def get_by_path(root, items):
    """Access a nested object in root by item sequence."""
    return reduce(operator.getitem, items, root)

def set_by_path(root, items, value):
    """Set a value in a nested object in root by item sequence."""
    get_by_path(root, items[:-1])[items[-1]] = value

####
####

def kmer_dict_init(ksize= 3,bases='ATCG'):
    '''produce nested dictionary of nucs for a particular kmer size'''
    mut_lib= recursively_default_dict()

    ksize= 3
    base_set= [bases]*ksize

    for trimer in product(*base_set):
        set_by_path(mut_lib,list(trimer),0)
    
    return mut_lib


def fasta_get_freq(seq,start= 0,end= 0,step= 1,ksize=3,bases= 'ATCG'):
    '''return count of kmer across fasta region'''
    kmer_dict= kmer_dict_init(ksize= ksize,bases=bases)
    if end == 0:
        end= len(seq) - ksize
    
    for ki in range(start,end,step):
        kmer= seq[ki:ki+ksize] 
        get_by_path(kmer_dict, kmer[:-1])[kmer[-1]] += 1
    
    return kmer_dict
        


def get_complement(kmer):
    '''Return complement of a given kmer'''
    complements= {
        'A': 'T',
        'T': 'A',
        'C': 'G',
        'G': 'C'
    }
    
    comp= [complements[x] for x in kmer][::-1]
    return comp


def complement_dicts(bases= 'ACGT',ksize= 3):
    '''return dict of comp kmers + index dict to parse with'''
    comp_dict= {}
    comp_index= {}
    d= 0
    base_set= [bases] * ksize
    mers= product(*base_set)
    
    for kmer in mers:
        kmer= ''.join(kmer)
        if kmer not in comp_dict.keys():
            comp_dict[kmer]= d
            
            comp= get_complement(kmer)
            comp= ''.join(comp)
            comp_dict[comp]= d
            
            comp_index[d]= (kmer,comp)
            d += 1
    
    return comp_dict,comp_index


def collapse_freqs(kmer_dict,comp_index):
    '''return vector of collapsed counts by kmer'''
    counts= []
    
    for kdx in comp_index.keys():
        total= [get_by_path(kmer_dict, list(comp)) for comp in comp_index[kdx]]
        total= sum(total)
        counts.append(total)
        
    return counts
            
    
    
    


In [5]:
bases= 'ATCG'
ksize= 3

kmer_dict= fasta_get_freq(random_fasta,start= 0,end= 0,step= 1,ksize=ksize,bases= bases)
comp_dict, comp_index= complement_dicts(bases= bases,ksize= ksize)
collapsed_freqs= collapse_freqs(kmer_dict,comp_index)
collapsed_freqs= np.array(collapsed_freqs) / sum(collapsed_freqs)
comp_index

{0: ('AAA', 'TTT'),
 1: ('AAT', 'ATT'),
 2: ('AAC', 'GTT'),
 3: ('AAG', 'CTT'),
 4: ('ATA', 'TAT'),
 5: ('ATC', 'GAT'),
 6: ('ATG', 'CAT'),
 7: ('ACA', 'TGT'),
 8: ('ACT', 'AGT'),
 9: ('ACC', 'GGT'),
 10: ('ACG', 'CGT'),
 11: ('AGA', 'TCT'),
 12: ('AGC', 'GCT'),
 13: ('AGG', 'CCT'),
 14: ('TAA', 'TTA'),
 15: ('TAC', 'GTA'),
 16: ('TAG', 'CTA'),
 17: ('TTC', 'GAA'),
 18: ('TTG', 'CAA'),
 19: ('TCA', 'TGA'),
 20: ('TCC', 'GGA'),
 21: ('TCG', 'CGA'),
 22: ('TGC', 'GCA'),
 23: ('TGG', 'CCA'),
 24: ('CAC', 'GTG'),
 25: ('CAG', 'CTG'),
 26: ('CTC', 'GAG'),
 27: ('CCC', 'GGG'),
 28: ('CCG', 'CGG'),
 29: ('CGC', 'GCG'),
 30: ('GAC', 'GTC'),
 31: ('GCC', 'GGC')}

## Markov Chain

We will consider possible only transitions where middle nucleotide changes. And we will make the probability of remaining the same nucleotide pretty large. 


In [6]:
def isAmatch(kmer1,kmer2):
    '''are two kmers identical across flanking-to-middle nucs? return 1 if yes, 0 if not.'''
    
    middle= int(len(kmer1) / 2)
    compat= [kmer1[:middle] == kmer2[:middle],kmer1[middle+1:] == kmer2[middle+1:]]
    compat= sum(compat)
    
    return int(compat == 2)

def match_lists(kmers1,kmers2):
    '''flank compatibility across lists of kmers'''
    results= 0
    for km in kmers1:
        for kl in kmers2:
            results += isAmatch(km,kl)
    
    return results


def get_possible(idx,comp_index):
    '''return binary vector of possible transition states'''
    
    test= [int(match_lists(comp_index[idx],comp_index[x]) > 0) for x in comp_index.keys()]
    
    return test


In [7]:

def matrixFill_randomGamma(comp_index,alpha= 0.05):
    '''return transition matrix with gamma drawn values'''
    
    bin_matrix= [get_possible(x,comp_index) for x in comp_index.keys()]
    bin_matrix= np.array(bin_matrix,dtype= float)
        
    where_poss= np.where(bin_matrix == 1)
    where_poss= [[where_poss[0][x],where_poss[1][x]] for x in range(len(where_poss[0]))]
    
    for idx in where_poss:
        
        if idx[0] != idx[1]:
            
            new_val= 0.5
            while new_val >= 0.5:
                new_val= gamma.rvs(alpha, size=1)[0]
            bin_matrix[idx[0],idx[1]]= new_val
            
    
    rowSums= np.sum(bin_matrix,axis= 1)
    
    for row in range(bin_matrix.shape[0]):
        total= 2 - rowSums[row]
        bin_matrix[row,row]= total
    
    return bin_matrix.T
        
        

In [8]:
import time
Tmat= matrixFill_randomGamma(comp_index,alpha= 0.05)

f= list(collapsed_freqs)
burnin= 1000
previous= list(collapsed_freqs)

diff_tally= []

for run in range(burnin):
    #clear_output()

    
    f= f @ Tmat
    diff_tally.append(np.std(previous - f))
    previous= list(f)
    
    #time.sleep(1)


fig = [go.Bar(
    x= [comp_index[x][0] for x in comp_index.keys()],
    y= collapsed_freqs
)]
iplot(fig)


fig = [go.Bar(
    x= [comp_index[x][0] for x in comp_index.keys()],
    y= f
)]
iplot(fig)


In [9]:
fig= [go.Scatter(
    x= list(range(len(diff_tally))),
    y= diff_tally
)]


layout= go.Layout()


fig = go.Figure(data=fig, layout= layout)
iplot(fig)

In [10]:
from sklearn.cluster import MeanShift, estimate_bandwidth

a= 0.05
r = gamma.rvs(a, size=1000).reshape(-1,1)

bandwidth = estimate_bandwidth(r, quantile=0.2, n_samples=len(r))

X_plot = np.linspace(0,1,len(r))
kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(r)
log_dens = kde.score_samples(X_plot.reshape(-1,1))


fig= [go.Scatter(x=X_plot, y=np.exp(log_dens), 
                            mode='lines', fill='tozeroy',
                            line=dict(color='blue', width=2))]
##

layout= go.Layout()


fig = go.Figure(data=fig, layout= layout)
iplot(fig)

### Sequence evolution



In [11]:
##### Expected site frequency spectrum
#####
bases_probs= [.25] * 4
L= 10000
fasta_seq= np.random.choice(list(bases),size=L,p= bases_probs,replace= True)
fasta_seq= ''.join(fasta_seq)

mu= 1e-7
Ne= 20000
Theta= 4 * Ne * mu
Nsamp= 1092

freq_exp= [Theta / x for x in range(1,Nsamp)]
freq_exp= [1-sum(freq_exp)] + freq_exp
#freq_exp= np.array(freq_exp) / np.sum(freq_exp)
freqs_possible= [x for x in range(0,Nsamp)]

freq_sample= np.random.choice(freqs_possible,len(fasta_seq),p=freq_exp)
sum(freq_sample > 0)

607

In [12]:
from scipy.stats import binom

import math

def nCr(n,r):
    f = math.factorial
    return f(n) // f(r) // f(n-r)


def get_grid(P= 20,range_height= [0,1],range_length= [0,1]):

    traces= [x for x in it.product(range(P),range(P))]

    i_coords, j_coords = np.meshgrid(np.linspace(range_height[0],range_height[1],P),
                          np.linspace(range_length[0],range_length[1],P),indexing= 'ij')

    traces= [x for x in it.product(range(P),range(P))]

    i_coords, j_coords = np.meshgrid(np.linspace(range_height[0],range_height[1],P),
                          np.linspace(range_length[0],range_length[1],P),indexing= 'ij')

    background= np.array([i_coords, j_coords])

    background= [background[:,c[0],c[1]] for c in traces]
    background=np.array(background)
    
    return background,i_coords, j_coords


def single_gen_matrix(Ne= 1092, ploidy= 2,precision= 1092):
    '''
    define transition probabilities for alleles at any given frequency from 1 to Nsamp.
    '''
    pop_size= Ne * ploidy
    space= np.linspace(0,pop_size,precision,dtype=int)
    t= np.tile(np.array(space),(precision,1))
    
    probs= [x / (pop_size) for x in space]
    
    freq_matrix= binom.pmf(t.T,pop_size,probs).T
    
    return freq_matrix


Ne= 2000
precision= 1000
freq_matrix= single_gen_matrix(Ne= 60,precision= 60)
freq_matrix.shape

background,i_coords, j_coords= get_grid(P= freq_matrix.shape[0])

In [89]:
colors_prob= freq_matrix.T.reshape(1,np.prod(freq_matrix.shape))
import scipy

colors_prob= scipy.stats.norm(np.mean(colors_prob),np.std(colors_prob)).cdf(colors_prob)[0]
fig= [go.Scatter(
    x= background[:,0],
    y= background[:,1],
    mode= 'markers',
    marker= {
        'color':colors_prob,

        'colorscale':'Viridis',
        'line': {'width': 0},
        'size': 5,
        'symbol': 'circle',
      "opacity": 1
      }
)]

fig = go.Figure(data=fig)
iplot(fig)

In [90]:
def freq_progression(fr= 1,n_gens= 20,freq_matrix= {}):
    '''frequency distribution after n generations given initial freq'''
    
    if isinstance(fr,int):
        freq_ar= [0] * freq_matrix.shape[0]
        freq_ar[fr]= 1
    else:
        freq_ar= fr
    
    for idx in range(n_gens):
        freq_ar= freq_ar @ freq_matrix.T
        freq_ar= freq_ar / np.sum(freq_ar)
    
    return freq_ar


In [95]:
Ne= 20000
precision= 1000
freq_matrix= single_gen_matrix(Ne= Ne,precision= precision)

print(freq_matrix.shape)

(1000, 1000)


In [79]:
multiplier= precision / Nsamp

init_freq= np.linspace(1,10000,15,dtype=int)
init_freq= init_freq * multiplier
init_freq= np.array(init_freq,dtype= int)

gen_time= 10

freq_prog= [freq_progression(int(x),n_gens= gen_time, freq_matrix= freq_matrix) for x in init_freq]

fig= [go.Scatter(
    x= np.linspace(0,1,freq_matrix.shape[0]),
    y= freq_prog[x],
    name= str(init_freq[x])
) for x in list(range(len(init_freq)))]

layout= go.Layout(
    xaxis= dict(
        range= [-0.05,1.05]
    )
)

figure= go.Figure(data= fig,layout= layout)
iplot(figure)

In [102]:
gens= np.linspace(1,200,15,dtype= int)
freqi= 10100
freqi= int(freqi * multiplier)


freq_prog= [freq_progression(freqi,n_gens= x, freq_matrix= freq_matrix) for x in gens]

fig= [go.Scatter(
    x= np.linspace(0,1,freq_matrix.shape[0]),
    y= freq_prog[x],
    name= str(gens[x])
) for x in list(range(len(gens)))]

layout= go.Layout(
    xaxis= dict(
        range= [-0.05,1.05]
    )
)

figure= go.Figure(data= fig,layout= layout)
iplot(figure)